In [1]:
import pandas as pd
import numpy as np
from scipy import sparse

from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, StandardScaler, LabelEncoder
#from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin


from xgboost import XGBClassifier
import xgboost as xgb

from datetime import datetime
from datetime import date

import re
import pickle

In [2]:
from sklearn.neighbors import KNeighborsClassifier

In [3]:
from sklearn.metrics import accuracy_score, jaccard_score, recall_score, precision_score, f1_score, classification_report, confusion_matrix

In [4]:
import nltk

In [5]:
from nltk.corpus import stopwords                   #Stopwords corpus
from nltk.tokenize import word_tokenize

In [6]:
# import textblob
# from textblob import TextBlob

In [7]:
from string import punctuation

In [8]:
import requests,json

import csv
from datetime import datetime

#### read train and test data

In [9]:
df_train = pd.read_csv('C:/Projects/FCSD/FNOP/old/Traindata_pilot_sample_v0.csv', engine='python')

In [10]:
df_test = pd.read_csv('C:/Projects/FCSD/FNOP/Data EDA/Testdata_pilot_sample_v0_top40parts.csv', engine='python')

In [58]:
df_X = df_train[['mdl_yr','tis_wsd','milge', 'veh_line_cd', 'eng_cd']]
df_X_cat = df_train[['mdl_yr','veh_line_cd', 'eng_cd']]
df_X_cat['mdl_yr'] = df_X_cat['mdl_yr'].astype('object')

C:\Users\jluo51\AppData\Local\Continuum\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [59]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
ohe.fit(df_X_cat)
onehot_encoded_X = ohe.transform(df_X_cat)

In [60]:
cat_names = np.concatenate(ohe.categories_, axis = 0)

In [61]:
cat_df = pd.DataFrame(onehot_encoded_X, columns = cat_names)

In [62]:
reliability_df = pd.read_csv("f_rate_WB_118month_poc3.csv", engine='python')

In [63]:
reliability_df.columns = ['prt_num', 'veh_line_cd', 'mdl_yr', 'tis_wsd', 'Avg_Weibull_Failure', 'f_rate']

In [64]:
data_reliab_pivot = reliability_df.pivot_table(index=['veh_line_cd', 'mdl_yr', 'tis_wsd'], columns='prt_num', values=['Avg_Weibull_Failure'], aggfunc='first')
        
reliability_df_t = data_reliab_pivot.fillna(0.0)

In [65]:
df1 = pd.merge(df_X, reliability_df_t, how='left', on=['veh_line_cd', 'mdl_yr', 'tis_wsd'])

C:\Users\jluo51\AppData\Local\Continuum\Anaconda3\envs\tf\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [66]:
df_X_dummy = pd.concat([cat_df, df_X[['tis_wsd','milge']], df1.iloc[:,5:]], axis=1)

### Use universal sentence encoder to create doc embedding

In [11]:
lookup_table = 'Misspellings_Correction_Synonyms_Lookup_Table_Master.xlsx'

In [12]:
syn_df=pd.read_excel(lookup_table)
syn_df = syn_df[syn_df['Review'] != "D"]
syn_dict=dict(zip(syn_df.misspelled, syn_df.corrected))

In [13]:
def TextPreprocess(text):
        # Avoid mixed data type errors and apply str on all rows
        # convert to lower case
        text = str(text).lower()

        #Remove HTML Tags
        text = re.sub('<[^>]*>', '', text)
        r1 = re.compile('<.*?>')
        text = re.sub(r1, ' ', text)

#         # Remove any token with numbers and/or special characters
#         r2 = re.compile(r'^[0-9\W\_]+$')
#         text = re.sub(r2, ' ', text)
        
        # move parenthesis content to the end of paragraph
        paren_content_extract = re.compile(r'\(([^()]+)\)')
        p_cont = paren_content_extract.findall(text)
        text = re.sub(r'\([^()]*\)', ' ', text)
        text = text + '. '.join(p_cont) + '.'
        
        # hard code to convert abbreviations with \'s
        text = re.sub(r"what's", "what is ", text)
        text = re.sub(r"\'s", " ", text)
        text = re.sub(r"\'ve", "have not ", text)
        text = re.sub(r"can't", "can not ", text)
        text = re.sub(r"n't", " not ", text)
        text = re.sub(r"i'm", "i am ", text)
        text = re.sub(r"\'re", " are ", text)
        text = re.sub(r"\'d", " would ", text)
        text = re.sub(r"\'ll", " will ", text)
        text = re.sub(r"\'scuse", " excuse ", text)
        
        # if a token starts with numerical or # sign, remove it
        text = re.sub('\b[\d#]+\S*\b', ' ', text)
        # if a token ends with numberical, remove it
        text = re.sub(r'\b\S+\d+\b', ' ', text)

#         #Removing Punctuations
#         text = re.sub(r'[?|!|\'|"|#]',r'',text)
#         text = re.sub(r'[.|,|)|(|\|/]',r' ',text)     

        #Removing Punctuations
        text = re.sub(r'[\'|"|#]',r'',text)
        text = re.sub(r'[)|(]',r' ',text) 
        
        # whitespace characters are replaced into space, multiple dots replaced into single dot
        text = re.sub('\s+', ' ', text)
        text = re.sub(r'\.+', '.', text)

        # Tokenize all text data using nltk package
        text = word_tokenize(text)

#         # Remove stop words
#         stop = stopwords.words('english')
#         stop1 = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z', \
#                  'c/s','c/states','c/stated','cu', 'cs','c-s', 'cus','tomer', 'stomer', 'tate', 'cust',  'cust states', \
#                  'advise','nana','NaN','customer','states','rr','nan','wont','ck','check','pls','please', 'client','owner']

#         # Combine default nltk stop list and custom stop list to generate a deduplicated list
#         english_stops = list(set(stop+stop1))

#         text = [item for item in text if item not in english_stops]

#         # Remove special characters occuring as prefixes
#         text = [re.sub(r'^[\W\_]+','',str(i)) for i in text]
#         # Remove special characters occuring as suffixes
#         text = [re.sub(r'[\W\_]+$','',str(i)) for i in text]

#         # Replace special characters with spaces when occuring within the token 
#         #(i.e. not at the beginning or the end) only for tokens greater than 5 characters in length
#         text =[re.sub(r'[\W\_]+', ' ', str(i)) if len(str(i)) > 5 else i for i in text]

#         # Replace special characters with null when occuring within the token 
#         #(i.e. not at the beginning or the end) only for tokens less than 6 characters in length
#         text = [re.sub(r'[\W\_]+', '', str(i)) if len(str(i)) < 6 else i for i in text]

#         # Remove stop words after data cleansing steps to remove any residual stop words
#         text = [item for item in text if item not in english_stops]

#         # remove any numbers after data cleansing steps to remove any residual numeric values
#         r3 = re.compile(r'^[0-9\W\_]+$')
#         text = [i for i in text if not r3.match(i)]

        #Perform synonym lookup to find matching correctly spelled tokens for misspelled tokens
        #spell=SpellChecker() 
        text = [syn_dict.get(str(i), str(i)) for i in text]

        # Convert the list to string before passing to lemmatization module
        text = ' '.join(text)

#         # Perform Lemmatization on the cleaned up data before tfidf vectorization
#         text = lemmatize_with_postag(text)
        text = text.replace(' .', '.').replace(' ?', '?').replace(' !', '!').replace(' ,', ',').replace(' /', '/')
        return(text)

In [14]:
text_df = df_train[['cust_txt']]

In [15]:
text_df['process_txt']=text_df['cust_txt'].apply(lambda x: TextPreprocess(x))

C:\Users\jluo51\AppData\Local\Continuum\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
### convert null values of customer_txt into ' '
text_df.loc[text_df['process_txt'].isnull()==True,'process_txt'] = ' '

In [13]:
import tensorflow as tf
import tensorflow_hub as hub

In [14]:
### load the model
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
use_model = hub.KerasLayer(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [15]:
sentence_embeddings_X = use_model(text_df['process_txt']).numpy()

In [18]:
column_names = ['col'+str(i) for i in range(512)]

In [19]:
cust_text_embedding = pd.DataFrame(sentence_embeddings_X, columns = column_names)

In [20]:
cust_text_embedding.shape

(358000, 512)

#### create label encoded y variables

In [21]:
le = LabelEncoder()
le.fit(df_train['target'])
label_encoded_y = le.transform(df_train['target'])

### train with KNN model

In [22]:
k = 5
knn_1 = KNeighborsClassifier(n_neighbors = k, metric = 'cosine')
knn_1.fit(cust_text_embedding,label_encoded_y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='cosine',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [23]:
### get prediction of the train data
y_train_predicted = knn_1.predict(cust_text_embedding)

In [24]:
accuracy_score(label_encoded_y, y_train_predicted)

0.7433268156424581

#### one-hot encode the predicted result from knn

In [30]:
ohe_knn_label = LabelBinarizer()

In [31]:
ohe_knn_label.fit(range(max(label_encoded_y)+1))
onehot_encoded_knn_label = ohe_knn_label.transform(y_train_predicted)

In [68]:
onehot_encoded_knn_label_df = pd.DataFrame(onehot_encoded_knn_label)

In [70]:
df_X_dummy_1 = pd.concat([df_X_dummy, onehot_encoded_knn_label_df], axis=1)

##### split this onehot encoded knn label and then use XGB to train a new model

In [71]:
X_train, X_eval, y_train, y_eval = train_test_split(df_X_dummy_1, label_encoded_y, test_size = 0.2, random_state = 11)

In [72]:
dtrain = xgb.DMatrix(X_train, y_train)
deval = xgb.DMatrix(X_eval, y_eval)

In [73]:
eval_set = [(dtrain, 'train'), (deval, 'eval')]

In [74]:
num_class = len(set(df_train['target']))
num_class

121

In [75]:
params = {
    "objective": "multi:softmax",
    "num_class": num_class,
    ##"eval_metric": "auc",
    "booster": "gbtree",
    "tree_method": "gpu_hist",
    "eta": 0.1,
    "max_depth": 4,
    "sub_sample": 1,
    "col_sample_bytree": 1,
    "verbosity": 0,    
    "seed": 11    
}

In [76]:
num_rounds = 100
early_stopping_rounds = 10

In [77]:
gbm = xgb.train(params, dtrain, num_rounds, evals=eval_set, early_stopping_rounds=early_stopping_rounds)

[0]	train-mlogloss:1.76214	eval-mlogloss:1.77275
[1]	train-mlogloss:1.65417	eval-mlogloss:1.66757
[2]	train-mlogloss:1.57794	eval-mlogloss:1.59332
[3]	train-mlogloss:1.51589	eval-mlogloss:1.53296
[4]	train-mlogloss:1.46379	eval-mlogloss:1.48241
[5]	train-mlogloss:1.41869	eval-mlogloss:1.43877
[6]	train-mlogloss:1.37928	eval-mlogloss:1.40073
[7]	train-mlogloss:1.34457	eval-mlogloss:1.36704
[8]	train-mlogloss:1.31329	eval-mlogloss:1.33689
[9]	train-mlogloss:1.28529	eval-mlogloss:1.30987
[10]	train-mlogloss:1.25976	eval-mlogloss:1.28542
[11]	train-mlogloss:1.23666	eval-mlogloss:1.26327
[12]	train-mlogloss:1.21510	eval-mlogloss:1.24298
[13]	train-mlogloss:1.19567	eval-mlogloss:1.22458
[14]	train-mlogloss:1.17749	eval-mlogloss:1.20740
[15]	train-mlogloss:1.16085	eval-mlogloss:1.19185
[16]	train-mlogloss:1.14549	eval-mlogloss:1.17754
[17]	train-mlogloss:1.13137	eval-mlogloss:1.16439
[18]	train-mlogloss:1.11811	eval-mlogloss:1.15212
[19]	train-mlogloss:1.10588	eval-mlogloss:1.14083
[20]	train

#### create X_test and y_test

In [78]:
df_X_test = df_test[['mdl_yr','tis_wsd','milge', 'veh_line_cd', 'eng_cd']]
df_X_test_cat = df_test[['mdl_yr','veh_line_cd', 'eng_cd']]
df_X_test_cat['mdl_yr'] = df_X_test_cat['mdl_yr'].astype('object')

C:\Users\jluo51\AppData\Local\Continuum\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [79]:
onehot_encoded_X_test = ohe.transform(df_X_test_cat)
cat_df_test = pd.DataFrame(onehot_encoded_X_test, columns = cat_names)

In [80]:
df_with_reliability = pd.merge(df_X_test, reliability_df_t, how='left', on=['veh_line_cd', 'mdl_yr', 'tis_wsd'])

C:\Users\jluo51\AppData\Local\Continuum\Anaconda3\envs\tf\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [81]:
df_X_test_dummy = pd.concat([cat_df_test, df_X_test[['tis_wsd','milge']], df_with_reliability.iloc[:,5:]], axis=1)

In [41]:
text_df_test = pd.read_csv('C:/Projects/FCSD/FNOP/Data EDA/Testdata_pilot_sample_v0_top40parts_process_text_sentEnc.csv')

In [42]:
### convert null values of customer_txt into ' '
text_df_test.loc[text_df_test['process_txt'].isnull()==True,'process_txt'] = ' '

In [43]:
sentence_embeddings_X_test = use_model(text_df_test['process_txt']).numpy()

In [44]:
cust_text_embedding_test = pd.DataFrame(sentence_embeddings_X_test, columns = column_names)

In [45]:
cust_text_embedding_test.shape

(84733, 512)

##### use trained knn model to encode the customer text, and then combine this encode with other tabular vehicle features

In [46]:
y_test_predicted = knn_1.predict(cust_text_embedding_test)

In [47]:
onehot_encoded_knn_label_test = ohe_knn_label.transform(y_test_predicted)

In [82]:
onehot_encoded_knn_label_test_df = pd.DataFrame(onehot_encoded_knn_label_test)

In [83]:
df_X_test_dummy_1 = pd.concat([df_X_test_dummy, onehot_encoded_knn_label_test_df], axis=1)

#### create label encoded y variables for the test set

In [48]:
label_encoded_y_test = le.transform(df_test['target'])

#### score the test set with the trained knn model

In [84]:
y_pred = gbm.predict(xgb.DMatrix(df_X_test_dummy_1))

In [55]:
len(y_pred)

84733

In [86]:
accuracy_score(label_encoded_y_test, y_pred)

0.7338109119233357